In [1]:
from queue import PriorityQueue, Queue
import numpy as np
import random as rnd

In [5]:
class Cell: 
    def __init__(self, row, col, prev = None):
        self.row = row
        self.col = col
        self.prev = prev
    def __lt__(self, other):        
        if (self.row == other.row):
            return self.col < other.col
        return self.row < other.row
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col and self.prev is other.prev
    def __str__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + "NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")"
    def __repr__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + "NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")"

In [40]:
class Maze:
    # -1 is barrier, 0 is free
    def __init__(self, dim, p):
        self.board =  np.zeros([dim, dim], dtype=int)
        self.dim = dim
        obstaclesList = []
        for i in range(0, dim):
            for j in range(0, dim):            
                if(rnd.random() < p):
                    self.board[i, j] = -1
                    obstaclesList.append((i, j))
        self.obstacles = np.array(obstaclesList)
        self.board[0, 0] = 0
        self.board[dim - 1, dim - 1] = 0
    def isSolvable(self):
        return pathExists(self)
    def startFire(self):
        i = rnd.randint(0,self.dim-1)
        j = rnd.randint(0,self.dim-1)
        while(self.board[i][j] == -1 or (i==0 and j == 0) or (i == self.dim-1 and j == self.dim-1)):
            i = rnd.randint(0,self.dim-1)
            j = rnd.randint(0,self.dim-1)
        self.board[i][j] = 2

In [4]:
def pathExists(maze, fromCoords = (0,0), toCoords = None):
    if toCoords is None:
        toCoords = (maze.dim - 1, maze.dim - 1)
    toRow, toCol = toCoords
    visited = []
    fringe = Queue()
    fringe.put(fromCoords)
    while not fringe.empty():
        currentCoords = fringe.get()
        currentRow, currentCol = currentCoords
        if (currentRow == toRow and currentCol == toCol):
            return True
        else:
            neighbors = findNeighboringCoords(currentCoords, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                fringe.put(neighbor)
            visited.append(currentCoords)
    return False

In [42]:
def findNeighboringCoords(coords, maze):
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row][col] != 0):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

def findNeighboringCells(cell, maze):
    neighboringCoords = findNeighboringCoords((cell.row, cell.col), maze)
    neighbors = []
    for neighbor in neighboringCoords:
        row, col = neighbor
        neighbors.append(Cell(row, col, cell))
    return neighbors

In [6]:
def uniformCost(maze, cell):
    return 1

def shortestPathSearch(maze, startCoords = (0, 0), heuristicFunction = uniformCost):
    startRow, startCol = startCoords
    startCell = Cell(startRow, startCol)
    visited = []
    fringe = PriorityQueue()
    fringe.put((0, startCell))
    while not fringe.empty():
        pathLength, currentCell = fringe.get()
        if (currentCell.row == maze.dim - 1 and currentCell.col == maze.dim - 1):
            shortestPath = []
            while (currentCell != startCell):
                shortestPath.append((currentCell.row, currentCell.col))
                currentCell = currentCell.prev
            shortestPath.append((startRow, startCol))
            shortestPath.reverse()
            return shortestPath
        else:
            neighbors = findNeighboringCells(currentCell, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                nextPathLength = pathLength + heuristicFunction(maze = maze, cell = neighbor)
                fringe.put((nextPathLength, neighbor))
            visited.append(currentCell)
    return None

In [6]:
cell = Cell(0, 0)
cell2 = Cell(0, 1, cell)
print(cell)
print(cell2)

(0, 0) :: prev=NONE


NameError: name 'prev' is not defined